# Create and display Landsat Mosaics

## Installation

```
$ pip install requests mapboxgl
```

In [ ]:
import os
import json
import random
import requests
import datetime
import urllib.parse

from mapboxgl.utils import *
from mapboxgl.viz import *

token = os.environ["MAPBOX_ACCESS_TOKEN"]

endpoint = "https://landsatlive.live"


In [ ]:
bounds = [-6.17431640625, 42.79540065303723, 8.28369140625, 51.0275763378024]

geojson = {
    'type': 'Feature',
    'geometry': {
        'type': 'Polygon',
        'coordinates': [[
            [bounds[0], bounds[1]],
            [bounds[2], bounds[1]],
            [bounds[2], bounds[3]],
            [bounds[0], bounds[3]],
            [bounds[0], bounds[1]]]]},
    'properties': {}
}

In [ ]:
viz = LinestringViz(geojson,
    access_token=token,
    line_width_default=4,
    center=((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2),
    zoom=4,
)
viz.show()

## Create Simple Mosaic

In [ ]:
# STAC QUERY (sat-api)
# Cloud Filters
min_cloud = 0
max_cloud = 20

# Date Filters
date_min="2013-01-01"
date_max="2019-12-01"
start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "bbox": bounds,
    "time": f"{start}/{end}",
    "query": {
        "eo:sun_elevation": {"gt": 0},
        "landsat:tier": {"eq": "T1"},
        "collection": {"eq": "landsat-8-l1"},
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud}
    },
    "sort": [{
      "field": "eo:cloud_cover",
      "direction": "asc"
    }],
}

# We post the query to the mosaic endpoint
# `optimized_selection: True` is an option to force only one PATH-ROW scene per tile
# Because we will use `pixel_selection:"first"` this will minimized the number of image fetched
results = requests.post(
    f"{endpoint}/mosaic/create", 
    json=query,
    params = {
        "optimized_selection": True,
        "seasons": "summer" # this should be a "," separated list of seasons
    },
).json()

# Viz
# Tiles Options
query_params = dict(
    bands="4,3,2", # True Color RGB
    color_ops="gamma RGB 3.5, saturation 1.7, sigmoidal RGB 15 0.35", # Looks nice
    pixel_selection="first",
)

# Create tile url
tiles_url = results["tiles"][0] + urllib.parse.urlencode(query_params)

viz = RasterTilesViz(
    tiles_url, 
    zoom=results["minzoom"],
    access_token=token,
    tiles_size=256,
    tiles_bounds=results["bounds"],
    center=results["center"],
    tiles_minzoom=results["minzoom"],
    tiles_maxzoom=results["maxzoom"],
)
viz.show()

## Create Simple Mosaic but sort cloud with highest first

In [ ]:
# STAC QUERY (sat-api)

# Cloud Filters
min_cloud = 0
max_cloud = 20

# Date Filters
date_min="2013-01-01"
date_max="2019-12-01"
start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "bbox": bounds,
    "time": f"{start}/{end}",
    "query": {
        "eo:sun_elevation": {"gt": 0},
        "landsat:tier": {"eq": "T1"},
        "collection": {"eq": "landsat-8-l1"},
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud}
    },
    "sort": [{
      "field": "eo:cloud_cover",
      "direction": "desc"
    }],
}

# We post the query to the mosaic endpoint
# `optimized_selection: True` is an option to force only one PATH-ROW scene per tile
# Because we will use `pixel_selection:"first"` this will minimized the number of image fetched
results = requests.post(
    f"{endpoint}/mosaic/create", 
    json=query,
    params = {
        "optimized_selection": True,
        "seasons": "summer" # this should be a "," separated list of seasons
    },
).json()

# Tiles Options
query_params = dict(
    bands="4,3,2", # True Color RGB
    color_ops="gamma RGB 3.5, saturation 1.7, sigmoidal RGB 15 0.35", # Looks nice
    pixel_selection="first",
)

# Create tile url
tiles_url = results["tiles"][0] + urllib.parse.urlencode(query_params)

viz = RasterTilesViz(
    tiles_url, 
    zoom=results["minzoom"],
    access_token=token,
    tiles_size=256,
    tiles_bounds=results["bounds"],
    center=results["center"],
    tiles_minzoom=results["minzoom"],
    tiles_maxzoom=results["maxzoom"],
)
viz.show()